## 因子汇总

### 买卖单因子

* buy_ls_ratio_ret  

$ (BuyL_i / BuyS_i) * Ret_i$  
大小买单比率乘以收益率 某种程度上想刻画主力和散户的获利差距  
分组清晰均匀稳定 ir=0.5

* buy_per_ret_log  

$ log(\frac{B_i}{B_{i-1}} * Ret_i)$   

* exce_buy_l_log  

$ log((B_i - B_{i-1}\frac{Vol_i}{Vol_{i-1}})) $  

* netbuy_per_log_demed  

$f = log(\frac{B_i - S_i}{B_{i-1} - S_{i-1}})$  
$f=|(f - f.median())| / f.std() $   
减去中位数取绝对值是想改变因子的分布 下面同理

* buy_ret_act_trans   

$ f = BAct_i * Ret_i$  
$ f = |(f-f.median())| / f.std()$

- bsratio_morn30per_prod_log  

$ bsratio = B_i / S_I$  
$ morn30 = \text{开盘30min成交量} / \text{当日成交量}$  
$ f = np.log(bsratio * morn30)$


- buy_l_order_act_mktovermean  

$mktovermean = 1\; if\; mkt_{stock} > mkt_mean \;else\; NaN$    
$f = BAct * mktovermean$  
mkt：某日的市值取对数；

- buy_l_act_r10_mean_std_ratio  

$ BuyLAct$主动买入大单量，日频，取对数，后仍记作$BuyLAct$
$factor = BuyLAct.rolling(10).mean() / BuyLAct.rolling(10).std()$  
或者分子直接使用BuyLAct

- buy_l_order_act_demean  

$factor = BuyLAct / BuyLAct.rolling(10).mean()$  
很简单的形式但是还挺好

- buy_mean_devide_std_r20  

$BuyL.rolling(20).mean() / BuyL.rolling(20).std()$  
或者把mean换成max

- buy_ls_per_ratio_ret_log  



- buy_ls_ratio_ret

$(BuyL / BuyS)*Ret$

- sell_l_ret_log  

$log(SellL * Ret)$    
小于0的值直接设为NaN,空值虽然多，但是效果挺显著，也可以直接先取对数再乘Ret,有效减少空值数，效果还不错  
log(SellL * -Ret)效果也不错  

- exce_buy_abs_log

$ExceBuy = Buy_i - Buy_{i-1} * \frac{Vol_i}{Vol_{i-1}}$  
$factor = log(abs(ExceBuy))$

- buy_sell_square_sum_r10mean  

$ (BuyL^2 + Sell^2).rolling(10).sum()$

- buy_l_s_ratio_r10mean  

$ (BuyL/BuyS).rolling(10).mean()$

- buy_vol_log_r10_ols_slope

取$BuyL$和 $Vol$ rolling10日的数据，返回线性回归的斜率

### 价量

- vol_ret_over1std_per  

分钟频数据， 每日分钟频收益率偏离1sigma的成交量占比  
$Ret_{s,T,t}$表示单只股票s在日期T内,t分钟的收益率,用分钟的收盘价, $Vol_{s, T, t}$同理, 并记Vol_{s, T}为其向量  
    
将$Ret_{s,T,t}$按照分钟标准化，得到的序列仍记做$Ret_{s,T,t}$  
I = $Ret_{s,T,t}$.apply(lambda x: 1 if abs(x) > 1 else 0)  
$factor_{sT} = (I * Vol_{s, T}).sum() / Vol_{s, T}.sum()$向量点乘




- p_log_vol_dis_std_r10mean  

成交量在价格上的分布  
stockdaily为某只股票单日分钟级价格成交量数据  
stockdaily = stockdaily.set_index('price')['vol']  
fa = log(stockdaily).std()

- morn_afno_30_ret_r20corr  

取20天的早盘30分钟的收益率和尾盘30分钟的收益率做corr

- afno30_ret_cvar_neg_09_r20.csv  

$Afno30Ret:$尾盘30min收益率  
$factor = Afno30Ret.rolling(20)$中选取最小的两个值求平均

- volmin_log_zs_skew  

分钟级别的成交量数据, 取对数(记log(0)=NaN)，对单只股票做zscore之后返回偏度

- volmin_log_zs_morn30_devide_tail30_mean

```python
def f(x):
    x = ((x.T - x.mean(axis=1)) / x.std(axis=1)).T
    return x[x > 1].mean() / (x[x<-1].mean() + 1e-5)
volmin_log_zs_oer1std_devide_lower1std =\
     volmin_log.groupby(level='Date').progress_apply(lambda x: f(x))

- 成交量在价格上的分布、成交量在收益率上的分布，收益率在成交量上的分布....

### 筹码分布

- below_pcloday_vol_per  
计算日被小于收盘价的买入力量（高于or对比开盘价）
```python
pret = pmin / pmin.iloc[-1]
pret[pret > 1] = 0 #只计算低于收盘价的时刻
volmin = np.log(volmin)
factor = np.dot(volmin, pret) / volmin.sum()
factor = factor.rolling(5).mean()


- last10ret_last10vol_per  
尾盘拉升获利情况  
在9:30 1m_vwap上表现显著

last10ret=尾盘10分钟收益率  
last10vol_per:尾盘10分钟成交量在整天成交量的占比, 分钟成交量数据取对数  
factor = last10ret * last10vol_per

### 拥挤度因子

- 傅里叶变化将时序数据转化为频谱数据

- volmin_fft_max10_mean_log
分钟成交量 快速傅里叶变换 取强度前十的信号做均值,最后取对数调整分布

```python
def handle_stockly_daily(data_stock_daily:pd.Series):
        vol = data_stock_daily['LastVolume']
        vol = vol.iloc[:-3] #去掉尾盘竞价
        N = 237
        fft_res = np.fft.fft(vol)
        fft_freq = np.fft.fftfreq(N, 1/N)
        res = pd.Series(np.abs(fft_res)[:N//2], index=fft_freq[:N//2])
        res = res.sort_values(ascending=False)

        return res.iloc[:10].mean()

- volmin_std_fft_over_mean1std_inv_std_ratio

vol为分钟频成交量去掉前30min和最后3min  
vol做Fourier变换后，保留振幅强度前十的信号再做逆变换得到vol_new  
factor = vol.std() / vol_new.std()
```python
def handle_stockly_daily(data_stock_daily:pd.Series):
        vol = data_stock_daily.iloc[30:-3]
        N = len(vol)
        vol.index = list(range(N))
        yf = pd.Series(np.fft.fft(vol))
        fft_freq = np.fft.fftfreq(N, 1/N)

        res = pd.Series(np.abs(yf)[:N//2], index=fft_freq[:N//2])
        res = res.sort_values(ascending=False)
        res_drop0 = res.iloc[1:].sort_values(ascending=False)
        idx = res_drop0.index[:10]

        yf2 = pd.Series(np.zeros(len(yf)))
        yf2.loc[idx] = yf.loc[idx]
        vol_new = pd.Series(np.real(np.fft.ifft(yf2)))

        return vol.std() / (vol_new.std() + 1e-5)

return data_stock.groupby(level='Date').apply(handle_stockly_daily)

### 非对称性因子
（未筛选）

- skew偏度因子  
$$skew = E[\frac{X - \mu}{\sigma}]$$
remark:标准化后的三阶矩，受到离群值的影响,实际上关系到了尾部分布，考虑去掉离群值？  

- 其他类skew的因子  
考虑做标准化的分子上减去的不是均值，而是中位数，四分位数等等  
$$skew = \frac{\mu - v}{E[|X-v|]}$$

- 顺序统计量  
$$ \lambda_r :=  \frac{1}{r}\sum_{k=0}^{r-1}(-1)^k \binom{r-1}{k}E[X_{r-k:r}]$$
其中$X_{k:n}$表示n个样本第k顺序统计量  
$$ L-skew = \lambda_3 / \lambda_2$$

- cvar尾部分布因子

- 密度函数估计  
熵估计，度量密度函数之间的距离，涉及对密度函数的估计，时间复杂度大  

- $Asym_p$  
考虑密度函数f和累积分布函数F之间的采样corr
$$Asym_p = -corr(f, F)$$
其中累计分布函数F由f积分得到  
idea:用chebyshev多项式逼近进行数值求积，即矩阵计算，速度更快

### 更多考虑

- $\textbf{增加即时项}$ 
研报中的因子构建都是基于向前rolling一段时间(记住T)的数据，每个数据在计算中的地位相当。因此磨灭了数据的时序特征，因子在t时间的因子值代表的是$[t-T+1,t]$这一段时间的分布的非对称性或者尾部特征，滞后性较强(特别地，对于尾部特征的因子常常出现一段时间因子值不变的情况)，

在我们的假设下，$[t-T+1,t]$的数据是随机变量$X_t$的$n$个样本，因此我们可以用这些数据来估计分布的性质，为了减弱计算的滞后性，我们通常需要减少窗口值$T$,但是也使得估计的模糊性加大

当因子值$f_t$与$f_{t-1}$)相同时，以尾部特征因子CVaR为例，说明样本$x_t$在样本$x_{t-T}-x_{t-1}$中较为居中，尾部特征较差。我觉得应该视其为一个后验的信息，对原先计算的因子值做出调整

当$f(t) < f(t-1)$时说明在$t-T$时刻出现一个尾部值，或者t时刻的尾部值小于T时刻，而在经过了$T$时间后数据t-T后依然居于尾部，说明分布的尾部性较差，应该惩罚减小f(t)值，且惩罚系数性格大于相同的情形；当$f(t)>f(t-1)$时，说明t时刻出现了一个尾部值，说明分布的尾部性较好，应该适当奖励增大$f(t)$

于是我们需要找到一个激活函数$s(x)$,使$x\leq0$时$s(x)<0$, $x>0$时$s(x)>0$
考虑选取最简单的分段函数：
$$s(x) =\left\{
\begin{array}{ll}
     x&if\quad x>0  \\
     -0.5&if \quad x=0\\
     -1+x&if \quad x<0
\end{array}
\right.
$$
则：
$$f_t = f(t)+\lambda s(f(t) - f(t-1))*f(t)$$
其中$\lambda$是惩罚系数，初步考虑设置在0.05左右  
想法本质上来源于想增加样本$x_t$在计算因子值$f_t$的权重